In [1]:
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np
import os

hv.extension('bokeh')

## Select the reservoir

In [2]:
RESERVOIR = '1078'

In [3]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-pts.geojson'))
val_polys = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/100-validation-reservoirs-grand-polys.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()  # select all 100 reservoirs
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name'] # dictionary that can be queried to get reservoir name

RESERVOIR_NAME = res_names[RESERVOIR]

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

# get reservoir properties from GRanD
nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area

min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area

area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])


## Plot a map of the selected reservoirs
global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

print(
    f"Selected reservoir: {RESERVOIR}: {RESERVOIR_NAME}\n",
    f"{nominal_area = }\n",
    f"{nominal_area_poly = }\n",
    f"{max_area = }\n",
    f"{min_area = }\n",
    f"{area_rep = }\n",
    f"{dam_height = }\n",
    f"{elev_msl = }\n",
    f"{depth = }\n",
    f"{capacity = }\n",
)

global_map

Selected reservoir: 1078: Lake Mead, US
 nominal_area = 580.95
 nominal_area_poly = 580.95
 max_area = nan
 min_area = 0
 area_rep = 634.5
 dam_height = 223.0
 elev_msl = 372.0
 depth = 63.2
 capacity = 36700.0



:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

<!-- ## Storage Calculation -->

In [4]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [5]:
srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/srtm_extrapolated/')

In [6]:
import numpy as np


if dam_height == -99:
    dam_height = np.nan
if elev_msl == -99:
    elev_msl = np.nan

aec_fp = Path(f'/tiger1/pdas47/tmsosPP/data/aec/srtm/{RESERVOIR}.csv')
aec = pd.read_csv(aec_fp)

Inspect elevations around the dam locations.

In [7]:
import ee
ee.Initialize()

| QUICK |: Change the `reservoir` variable to select poorly performing reservoirs.

In [40]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/tiger1/pdas47/tmsosPP/models/RAT/src")

from rat.ee_utils.ee_aec_file_creator import dem_percentile, aec_file_creator

# ?dem_percentile

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
lat = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['LAT_DD'].values[0]
lon = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['LONG_DD'].values[0]

# dem_percentile(
#     lat=lat,
#     lon=lon,
#     buffer_distance=250
# )

In [43]:
val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]

,index,GRAND_ID,RES_NAME,DAM_NAME,ALT_NAME,RIVER,ALT_RIVER,MAIN_BASIN,SUB_BASIN,NEAR_CITY,...,grand_id,rid_filepath,resops_id,rid_filename,tmsos_id,distance,COUNTRY_SHORT,name_2,climate,geometry
97,96,5796,Noi,Sirindhorn,None,Lam Dom Noi,None,Mekong,None,None,...,5796.0,mekong_insitu/200212-Sirindhorn_Dam.csv,NaN,200212-Sirindhorn_Dam.csv,0810,NaN,Th,"Noi, Th",3,"POLYGON ((105.37056 14.95278, 105.37065 14.952..."


In [44]:
def get_obs_aec_above_water_surface(aec, max_height):
    """
    Filters and processes the AEC (Area-Elevation Curve) data to obtain observations above a specified water surface height.

    Args:
        aec (pd.DataFrame): DataFrame containing AEC data with 'Elevation' and 'CumArea' columns.
        max_height (float): The maximum height of the water surface to filter the observations.

    Returns:
        pd.DataFrame: A DataFrame containing the filtered and processed AEC data with 'Elevation' and 'CumArea' columns.
    """
    obs_aec_above_water = aec[aec['Elevation'] < max_height]
    obs_aec_above_water = obs_aec_above_water.sort_values('Elevation')
    obs_aec_above_water['CumArea_diff'] = obs_aec_above_water['CumArea'].diff()
    obs_aec_above_water['z_score'] = (obs_aec_above_water['CumArea_diff'] - obs_aec_above_water['CumArea'].mean()) / obs_aec_above_water['CumArea'].std()
    max_z_core_idx = obs_aec_above_water['z_score'].idxmax()
    obs_aec_above_water = obs_aec_above_water.loc[max_z_core_idx:, :]
    obs_aec_above_water = obs_aec_above_water[['Elevation', 'CumArea']]

    return obs_aec_above_water


In [45]:
from scipy.integrate import cumulative_trapezoid

def calculate_storage(aec_df):
    """
    Calculate the storage of a reservoir from its Area-Elevation Curve (AEC).

    Parameters:
    aec_df (pd.DataFrame): DataFrame containing 'Elevation' and 'CumArea' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Storage' column representing the storage in cubic meters.
    """
    elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

    # cumulative_trapezoid takes two parameters.
    # y = y-axis locations of points. these values will be integrated. 
    # x = x-axis locations of points, where each y value is sampled. Area.
    storage = cumulative_trapezoid(
        elevation_normalized, 
        aec_df['CumArea'] * 1e6
    )
    storage = np.insert(storage, 0, 0)

    aec_df['Storage'] = storage
    aec_df['Storage (mil. m3)'] = storage * 1e-6
    return aec_df

In [46]:
from scipy.optimize import minimize

# Function to return predicted y-values from the polynomial
def predict_y(params, x):
    return np.polyval(params, x)

# Objective function: residuals to minimize
def objective(params, x, y):
    predicted_y = predict_y(params, x)

    return np.sum((predicted_y - y)**2)

# Constraint 1: dy/dx > 0 -> derivative of the polynomial should be positive for all x
def constraint_derivative(params, x):
    # Sample points across the entire range of x (0 to np.max(x))
    x_sample = np.linspace(0, np.max(x), 100)
    
    # Get the derivative coefficients of the polynomial
    derivative_coeffs = np.polyder(params[::-1], 1)  # Take the first derivative
    
    # Evaluate the derivative at these sample points
    derivative_values = np.polyval(derivative_coeffs, x_sample)  # np.polyder gives the derivative coefficients

    # Return the minimum value of the derivative; it should be greater than 0
    return derivative_values

# Constraint 2: intercept should be within (dam_bottom, dam_bottom + 5)
def constraint_intercept(params, dam_bottom):
    a0 = params[-1]  # Intercept is the last parameter (highest degree)
    return a0 - dam_bottom, (dam_bottom + 5) - a0

# Function to perform the minimization
def fit_polynomial(x, y, degree, dam_bottom):
    # Initial guess for the polynomial parameters using np.polyfit
    initial_guess = np.polyfit(x, y, degree)
    print("Initial guess: ", initial_guess[::-1])
    
    # Set up constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda params: constraint_derivative(params, x)},  # dy/dx > 0
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params, dam_bottom)[0]},  # intercept within lower bound
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params, dam_bottom)[1]}   # intercept within upper bound
    ]
    
    # Perform minimization
    result = minimize(objective, initial_guess, args=(x, y), constraints=constraints, options={'maxiter': 1000})
    
    # Print results
    print(f"Optimized polynomial for degree {degree}: {np.poly1d(result.x[::-1])}")
    return result, initial_guess

In [47]:
from pathlib import Path
from itertools import zip_longest,chain
from rat.ee_utils.ee_utils import poly2feature

BUFFER_DIST = 500
DEM = ee.Image('USGS/SRTMGL1_003')

def grouper(iterable, n, *, incomplete='fill', fillvalue=None):
    "Collect data into non-overlapping fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, fillvalue='x') --> ABC DEF Gxx
    # grouper('ABCDEFG', 3, incomplete='strict') --> ABC DEF ValueError
    # grouper('ABCDEFG', 3, incomplete='ignore') --> ABC DEF
    args = [iter(iterable)] * n
    if incomplete == 'fill':
        return zip_longest(*args, fillvalue=fillvalue)
    if incomplete == 'strict':
        return zip(*args, strict=True)
    if incomplete == 'ignore':
        return zip(*args)
    else:
        raise ValueError('Expected fill, strict, or ignore')

def _aec(n,elev_dem,roi, scale=30):
    ii = ee.Image.constant(n).reproject(elev_dem.projection())
    DEM141 = elev_dem.lte(ii)

    DEM141Count = DEM141.reduceRegion(
        geometry= roi,
        scale= scale,
        reducer= ee.Reducer.sum()
    )
    area=ee.Number(DEM141Count.get('elevation')).multiply(30*30).divide(1e6)
    return area

def get_obs_aec_srtm(aec_dir_path, scale, reservoir, reservoir_name):
    """
    Generates an observed Area-Elevation Curve (AEC) file for a given reservoir using SRTM data.
    The function checks if an AEC file already exists for the given reservoir. If it does, it reads the file and returns the data.
    If not, it generates the AEC using SRTM data. It saves the data as a csv file and returns the aecas a pandas dataframe.
    Parameters:
        aec_dir_path (str): The directory path where the AEC file will be saved.
        scale (int): The scale at which to perform the elevation calculations.
        reservoir (object): The reservoir object containing geometry information.
        reservoir_name (str): The name of the reservoir.
    Returns:
        pd.DataFrame: A DataFrame containing the elevation and cumulative area data.
    """
    print(f"Generating observed AEC file for {reservoir_name} from SRTM")
    aec_dst_fp = os.path.join(aec_dir_path,reservoir_name+'.csv')

    if Path(aec_dst_fp).exists():
        print(f"SRTM AEC exists for {reservoir_name} at {aec_dst_fp}")
        aec_df = pd.read_csv(aec_dst_fp, comment='#')
    else:
        reservoir_polygon = reservoir.geometry
        aoi = poly2feature(reservoir_polygon,BUFFER_DIST).geometry()
        min_elev = DEM.reduceRegion( reducer = ee.Reducer.min(),
                            geometry = aoi,
                            scale = scale,
                            maxPixels = 1e16,
                            bestEffort=True
                            ).get('elevation')
        max_elev = DEM.reduceRegion( reducer = ee.Reducer.max(),
                            geometry = aoi,
                            scale = scale,
                            maxPixels = 1e16,
                            bestEffort=True
                            ).get('elevation')

        elevs = ee.List.sequence(min_elev, max_elev, 1)
        elevs_list = elevs.getInfo()
        grouped_elevs_list = grouper(elevs_list,5)

        areas_list = []
        for subset_elevs_tuples in grouped_elevs_list:
                ## Removing None objects from grouped tuples and converting them to list and then ee.List and then calculating  area for that 
            subset_elevs_list = list(filter(lambda x: x != None, subset_elevs_tuples))
            subset_elevs = ee.List(subset_elevs_list)
            subset_areas = subset_elevs.map(lambda elevation_i: _aec(elevation_i, DEM, aoi, scale))
            subset_areas_list = subset_areas.getInfo()
                    ## Appendning areas to the main list
            areas_list.append(subset_areas_list)
                    
                ## Converting list of lists to list
        areas_list = list(chain(*areas_list))
        areas_list = np.round(areas_list,3)
                ## Preparing dataframe to write down to csv 
        aec_df = pd.DataFrame(
                    data = {
                        'Elevation' :elevs_list, 
                        'CumArea':areas_list
                        })
        aec_df.to_csv(aec_dst_fp,index=False)
        print(f"Observed AEC obtained from SRTM for {reservoir_name}")

    return aec_df

In [48]:
from shapely.geometry import Point

def dam_bottom_dem_percentile(
        dam_location, scale=30,
        percentiles=[1, 2, 3, 4, 5, 10], buffer_distance=1000,
        ee_dem_name = 'MERIT/DEM/v1_0_3'
    ):
    """
    Calculates elevations corresponding to percentile values within a buffered region around 
    a dam location.

    Parameters:
        dam_location (shapely.geometry.point.Point): The location of the dam as a point geometry.
        scale (int, optional): Scale in meters for the DEM data. Default is 30.
        percentiles (list of int, optional): List of percentiles to calculate. Default is [1, 2, 3, 4, 5, 10].
        buffer_distance (int, optional): Buffer distance in meters around the point of interest. Default is 1000.
        ee_dem_name (str, optional): Name of the Earth Engine DEM dataset. Default is 'MERIT/DEM/v1_0_3'.
    Returns:
        dict: A dictionary containing the calculated statistics (min, max, and specified percentiles).
    """
    lon, lat = dam_location.x, dam_location.y
    # Create a point geometry using the provided latitude and longitude
    point = ee.Geometry.Point([lon, lat])
    
    # Buffer the point to create a region of interest (ROI)
    feature = point.buffer(buffer_distance)
    # Load the DEM dataset
    dem = ee.Image(ee_dem_name)
    
    # Clip DEM to the buffered region (feature)
    dem_clipped = dem.clip(feature)
    
    # Define the reducer (min/max combined with percentiles)
    reducer = (ee.Reducer.minMax()
               .combine(ee.Reducer.percentile(percentiles), '', True))
    
    # Reduce the clipped DEM over the feature's geometry to get statistics
    stats = dem_clipped.reduceRegion(
        reducer=reducer,
        geometry=feature,
        scale=scale,  # Scale appropriate for MERIT DEM (30 meters)
        maxPixels=1e9
    )
    
    # Set the calculated statistics as properties of the feature
    return stats.getInfo()

def get_closest_point_to_dam(
        reservoir, dam_location, buffer_distance=90, 
        grwl_fp=Path('/tiger1/pdas47/tmsosPP/data/dam_bottom_elevation/GRWL/GRWL_summaryStats.shp')
    ):
    """
    Gets the closest point to the dam for a given reservoir.

    Parameters:
        tmsos_id (str): The tmsos_id of the reservoir.
        buffer_distance (int): The buffer distance around the reservoir in meters.

    Returns:
        tuple: (shapely.geometry.point.Point, matplotlib.figure.Figure or None)
               The closest point to the dam and the plot if PLOT is True, otherwise None.
    """
    grwl = gpd.read_file(grwl_fp)

    # Select the reservoir
    reservoir_df = reservoir

    # Clip GRWL by taking a 1 degree buffer around reservoir_df
    reservoir_df_buffered = reservoir_df.buffer(1)
    grwl_clipped = gpd.clip(grwl, reservoir_df_buffered)

    # Convert the CRS to the local UTM projection
    utm_crs = reservoir_df.estimate_utm_crs()
    grwl_utm = grwl_clipped.to_crs(utm_crs)
    reservoir_utm = reservoir_df.to_crs(utm_crs)

    # Take a buffer of buffer_distance meters around the grand reservoirs
    reservoir_utm_buffered = reservoir_utm.geometry.buffer(buffer_distance)

    # Convert buffered geometries back to GeoDataFrame
    reservoir_utm_buffered_gdf = gpd.GeoDataFrame(geometry=reservoir_utm_buffered, crs=utm_crs).reset_index(drop=True)

    # Reverse the clipping to keep the portion of GRWL polylines outside the reservoir polygon
    grwl_outside_reservoir = gpd.overlay(grwl_utm, reservoir_utm_buffered_gdf, how='difference')

    # Get the dam location in utm crs
    dam_location_geom = gpd.GeoDataFrame(geometry=[dam_location], crs=reservoir_df.crs).to_crs(utm_crs)

    # Convert the dam location and grwl_outside_reservoir back to lat/lon
    dam_location_latlon = dam_location_geom.to_crs(epsg=4326).geometry.iloc[0]
    grwl_outside_reservoir_latlon = grwl_outside_reservoir.to_crs(epsg=4326)

    # Find the closest point in the grwl_outside_reservoir polylines to the dam location
    closest_point_to_dam = grwl_outside_reservoir_latlon.geometry.apply(lambda geom: geom.interpolate(geom.project(dam_location_latlon)))
    closest_point_to_dam = closest_point_to_dam.iloc[closest_point_to_dam.distance(dam_location_latlon).idxmin()]

    return closest_point_to_dam, dam_location, grwl_clipped


def get_dam_bottom(reservoir, dam_location=None, grwl_fp=Path('/tiger1/pdas47/tmsosPP/data/dam_bottom_elevation/GRWL/GRWL_summaryStats.shp')):
    """
    Determines the dam bottom elevation for a given reservoir.

    Parameters:
        tmsos_id (str): The tmsos_id of the reservoir.
        grwl_fp (Path): The file path to the GRWL data.

    Returns:
        tuple: (float, str)
               The dam bottom elevation and the method used ('grwl_intersection' or 'dam_location').
    """
    # Load GRWL data
    grwl = gpd.read_file(grwl_fp)

    # Check if GRWL intersects with reservoir geometry
    intersection = grwl.intersects(reservoir.unary_union)

    if intersection.any():
        method = 'grwl_intersection'
        print("GRWL intersects with reservoir geometry.")
        closest_point_to_dam, _, _ = get_closest_point_to_dam(
            reservoir, dam_location,
            buffer_distance=90,  # 90 m buffer from reservoir = 3 pixels
            grwl_fp=grwl_fp
        )
        
        # Extract latitude and longitude of the closest point
        lat, lon = closest_point_to_dam.y, closest_point_to_dam.x
        
        # Calculate the dam bottom elevation using the dam_bottom_dem_percentile function
        stats = dam_bottom_dem_percentile(closest_point_to_dam, buffer_distance=250)
        
        # Extract the 1 percentile elevation as the dam bottom elevation
        dam_bottom_elevation = stats['dem_p1']
    else:
        method = 'dam_location'
        print("GRWL does not intersect with reservoir geometry.")
        
        # Calculate the dam bottom elevation using the dam_bottom_dem_percentile function with a 500 m buffer
        stats = dam_bottom_dem_percentile(dam_location, buffer_distance=500)
        
        # Extract the 1 percentile elevation as the dam bottom elevation
        dam_bottom_elevation = stats['dem_p1']

    return dam_bottom_elevation, method


reservoir = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]
# Extract latitude and longitude for the reservoir
lat_dd = reservoir['LAT_DD'].values[0]
long_dd = reservoir['LONG_DD'].values[0]

# Construct dam_location using the extracted latitude and longitude
dam_location = Point(long_dd, lat_dd)

get_dam_bottom(
    reservoir, dam_location
)

/tmp/ipykernel_1621489/3399360267.py:116: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersection = grwl.intersects(reservoir.unary_union)


GRWL intersects with reservoir geometry.


/tmp/ipykernel_1621489/3399360267.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reservoir_df_buffered = reservoir_df.buffer(1)
/tmp/ipykernel_1621489/3399360267.py:95: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_point_to_dam = closest_point_to_dam.iloc[closest_point_to_dam.distance(dam_location_latlon).idxmin()]


(114.06273651123048, 'grwl_intersection')

In [51]:

def extrapolate_reservoir(
    reservoir, dam_location, reservoir_name, dam_height, aec, aev_save_dir, buffer_distance=1000
):
    """
    Extrapolates the reservoir's Area-Elevation-Capacity (AEC) curve by fitting a polynomial to observed data and adds column for storage.
    - The function first calculates the dam bottom elevation using the specified percentile.
    - It then fits polynomials of degrees 3, 2, and 1 to the observed AEC data, stopping at the first successful fit.
    - The predicted AEC curve is generated and saved to a CSV file in the specified directory.
    - The initial guess and polynomial equation are stored as comments in the CSV file.
    
    Parameters:
        reservoir_name (str): Name of the reservoir.
        dam_height (float): Height of the dam.
        scale (float): Scale factor for the DEM data.
        dam_bottom_elevation_percentile (float): Percentile to determine the dam bottom elevation.
        aec (pd.DataFrame): DataFrame containing observed AEC data with columns 'CumArea' and 'Elevation'.
        aec_dir_path (str): Directory path to save the predicted AEC file.
    Returns:
        pd.DataFrame: DataFrame containing the predicted storage values with columns 'CumArea', 'Elevation', and 'Elevation_Observed'.
    """
    dam_bottom_elevation, method = get_dam_bottom(reservoir, dam_location) # from GRWL downstream point

    dam_top_elevation = dam_bottom_elevation + dam_height
    print(f"Dam bottom elevation for {reservoir_name} is {dam_bottom_elevation}")
    print(f"Dam top elevation for {reservoir_name} is {dam_top_elevation}")

    obs_aec_above_water = get_obs_aec_above_water_surface(
        aec, dam_top_elevation
    )

    x = obs_aec_above_water['CumArea']
    y = obs_aec_above_water['Elevation']

    # Try fitting polynomials starting from degree 3 down to 1
    for degree in [2, 1]:
        result, initial_guess = fit_polynomial(x, y, degree, dam_bottom_elevation)
        if result.success:
            print(f"Successfully fitted a degree {degree} polynomial for {reservoir_name}")
            break
        else:
            print(f"Failed to fit a degree {degree} polynomial for {reservoir_name}, trying lower degree")
    
    # Generate x_pred by combining a range of values from 0 to the maximum of x with the unique values from aec['CumArea']
    # and then sorting them. This ensures that x_pred includes all unique cumulative area values from the AEC data.
    x_pred = np.sort(np.unique(np.concatenate([np.arange(0, np.max(x), 0.25), obs_aec_above_water['CumArea'].values])))
    y_pred = predict_y(result.x, x_pred)

    # Create a new DataFrame with the predicted x and y values
    predicted_df = pd.DataFrame({
        'CumArea': x_pred, 
        'Elevation': y_pred
    })

    predicted_storage_df = calculate_storage(predicted_df)
    # Merge the predicted DataFrame with the observed AEC DataFrame to get the observed elevations
    predicted_storage_df = pd.merge(
        predicted_storage_df, 
        obs_aec_above_water[['CumArea', 'Elevation']], 
        on='CumArea', 
        how='left', 
        suffixes=('', '_Observed')
    )
    # Store the initial guess and result as comments in the AEC file
    with open(os.path.join(aev_save_dir, f"{reservoir_name}.csv"), 'w') as f:
        f.write(f"# Initial guess: {initial_guess[::-1]}\n")
        f.write(f"# Polynomial degree: {degree}\n")
        f.write(f"# dam bottom method: {method}")
        polynomial_equation = np.poly1d(result.x[::-1])
        for line in str(polynomial_equation).split('\n'):
            f.write(f"# {line}\n")
        predicted_storage_df.to_csv(f, index=False)
    print(f"Saved predicted AEC for {reservoir_name} to {os.path.join(aev_save_dir, f'{reservoir_name}.csv')}")

    return predicted_storage_df


In [52]:
reservoir_name = RESERVOIR_NAME
dam_height = reservoir['DAM_HGT_M'].values[0]
aec = get_obs_aec_srtm(
    '/tiger1/pdas47/tmsosPP/data/aec/srtm', 30, 
    reservoir.squeeze(), 
    RESERVOIR
)

Generating observed AEC file for 0810 from SRTM
SRTM AEC exists for 0810 at /tiger1/pdas47/tmsosPP/data/aec/srtm/0810.csv


In [54]:
from shapely.geometry import Point

aev_save_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/aev_2deg_poly_constraints_centerline_and_dam_buffer')

# Extract latitude and longitude for the reservoir
lat_dd = reservoir['LAT_DD'].values[0]
long_dd = reservoir['LONG_DD'].values[0]

# Create dam_location using the extracted latitude and longitude
dam_location = Point(long_dd, lat_dd)

extrapolated_aec = extrapolate_reservoir(
    reservoir, dam_location, RESERVOIR, dam_height, aec,
    aev_save_dir
)

/tmp/ipykernel_1621489/3399360267.py:116: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersection = grwl.intersects(reservoir.unary_union)


GRWL intersects with reservoir geometry.


/tmp/ipykernel_1621489/3399360267.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reservoir_df_buffered = reservoir_df.buffer(1)
/tmp/ipykernel_1621489/3399360267.py:95: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_point_to_dam = closest_point_to_dam.iloc[closest_point_to_dam.distance(dam_location_latlon).idxmin()]


Dam bottom elevation for 0810 is 114.06273651123048
Dam top elevation for 0810 is 156.06273651123047
Initial guess:  [ 8.06404541e+01  3.42209426e-01 -3.75513223e-04]
Optimized polynomial for degree 2:        2
114.1 x + 0.118 x - 5.985e-06
Successfully fitted a degree 2 polynomial for 0810
Saved predicted AEC for 0810 to /tiger1/pdas47/tmsosPP/data/aec/aev_2deg_poly_constraints_centerline_and_dam_buffer/0810.csv


In [39]:
extrapolated_aec.hvplot('CumArea', 'Elevation_Observed', kind='scatter') * \
extrapolated_aec.hvplot('CumArea', 'Elevation')

:Overlay
   .Scatter.I :Scatter   [CumArea]   (Elevation_Observed)
   .Curve.I   :Curve   [CumArea]   (Elevation)

In [ ]:
# import os

# def aec_file_creator(
#         reservoir_shpfile, shpfile_column_dict, aec_dir_path, 
#         scale=30, dam_bottom_elevation_percentile=1, 
#         # force_extrapolate=False, 
#         dam_buffer_distance=250
#     ):
#     # Obtaining list of csv files in aec_dir_path
#     aec_filenames = []
#     for f in os.listdir(aec_dir_path):
#         if f.endswith(".csv"):
#             aec_filenames.append(f[:-4])
#     if isinstance(reservoir_shpfile, str):
#         reservoirs_polygon = gpd.read_file(reservoir_shpfile)
#     elif isinstance(reservoir_shpfile, gpd.geodataframe.GeoDataFrame): 
#         reservoirs_polygon = reservoir_shpfile
#     else:
#         raise ValueError("reservoir_shpfile should be either a string or a geodataframe")
#     for reservoir_no,reservoir in reservoirs_polygon.iterrows():
#         # Reading reservoir information
#         reservoir_name = str(reservoir[shpfile_column_dict['unique_identifier']])
#         dam_height = float(reservoir[shpfile_column_dict['dam_height']])
#         dam_lat = float(reservoir[shpfile_column_dict['dam_lat']])
#         dam_lon = float(reservoir[shpfile_column_dict['dam_lon']])
#         # if (reservoir_name in aec_filenames) and (not force_extrapolate):
#         if (reservoir_name in aec_filenames):
#             print(f"Skipping {reservoir_name} as its AEC file already exists. Not extrapolating, force_extrapolate is turned off.")
#         elif (reservoir_name not in aec_filenames):
#             print(f"Generating new AEC for {reservoir_name}")
#             aec = get_obs_aec_srtm(aec_dir_path, scale, reservoir, reservoir_name)
            
#             # Check if 'Elevation_Observed' column exists in the AEC DataFrame, which means 
#             # extrapolation has already been done
#             # if 'Elevation_Observed' in aec.columns:
#             #     print(f"AEC for {reservoir_name} has already been extrapolated.")
#             # else:
#             # Replace the $SELECTION_PLACEHOLDER$ with a call to the new function
#             extrapolate_reservoir(
#                 reservoir_name, dam_lat, dam_lon, dam_height, scale, 
#                 dam_bottom_elevation_percentile, aec, aec_dir_path, 
#                 buffer_distance=dam_buffer_distance
#             )
#             # force_extrapolate = False # set extrapolation to false as it has been done in last step

#         # elif force_extrapolate:
#         #     aec = pd.read_csv(os.path.join(aec_dir_path,reservoir_name+'.csv'), comment='#')
#         #     print(f"Extrapolating AEC file for {reservoir_name} as force_extrapolate is turned on.")
#         #     # Replace the $SELECTION_PLACEHOLDER$ with a call to the new function
#         #     extrapolate_reservoir(
#         #         reservoir_name, dam_lat, dam_lon, dam_height, scale, 
#         #         dam_bottom_elevation_percentile, aec, aec_dir_path, 
#         #         buffer_distance=dam_buffer_distance
#         #     )

In [77]:
# from pathlib import Path

# aec_file_creator(
#     val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR],
#     {
#         'unique_identifier': 'tmsos_id',
#         'dam_height': 'DAM_HGT_M',
#         'dam_lat': 'LAT_DD',
#         'dam_lon': 'LONG_DD'
#     },
#     aec_dir_path = Path("/tiger1/pdas47/tmsosPP/data/aec/aev_2deg_poly_constraints_250m_buffer_around_dam"),
#     dam_buffer_distance=250,
# )

Skipping 0810 as its AEC file already exists. Not extrapolating, force_extrapolate is turned off.
AEC file exists for all reservoirs in this basin


1

In [15]:
# merit_dem_elevations = pd.read_csv("/tiger1/pdas47/tmsosPP/data/dam_bottom_elevation/MERIT_DEM_1000_m_around_dam_locations.csv", dtype={'tmsos_id': str}).drop(
#     ['system:index', '.geo'], axis=1
# )
# merit_dem_elevations.head()

,dem_max,dem_min,dem_p1,dem_p10,dem_p15,dem_p2,dem_p20,dem_p25,dem_p3,dem_p30,...,dem_p50,dem_p6,dem_p60,dem_p7,dem_p70,dem_p8,dem_p80,dem_p9,dem_p90,tmsos_id
0,1530.077271,541.943115,546.734673,550.019569,553.808512,546.734673,559.641687,559.641687,546.734673,559.641687,...,613.936004,550.019569,670.227516,550.019569,765.874281,550.019569,1033.159235,550.019569,1289.850638,0934
1,1464.071289,421.065369,422.092841,428.898907,435.753357,422.092841,443.648814,467.132673,422.092841,515.482880,...,667.396393,428.898907,780.639482,428.898907,891.923305,428.898907,1035.497431,428.898907,1155.850551,0936
2,1679.869873,573.878296,574.537821,731.259746,731.259746,579.384089,731.259746,740.009516,587.893527,771.838979,...,932.384094,644.621742,996.266236,667.734305,1100.172784,690.987614,1211.349972,724.447527,1340.358775,0930
3,1715.137085,446.809845,451.951807,516.658084,539.237789,451.951807,563.073492,571.347979,460.044878,571.347979,...,684.126697,475.978048,724.984660,475.978048,811.882498,491.161026,1052.369246,499.790276,1331.618650,0931
4,1172.023071,581.811462,589.834540,600.551163,618.154278,598.359481,626.191956,642.040255,600.551163,658.303761,...,757.818417,600.551163,854.034727,600.551163,954.043431,600.551163,1021.945790,600.551163,1085.982372,0933


In [16]:
# ## try to plot interactively to rapidly visualize 

# import panel as pn
# import hvplot.pandas
# pn.extension('bokeh')

# # Create a reservoir selector dropdown menu
# reservoir_selector = pn.widgets.IntSlider(name='Reservoir Selector', start=0, end=len(selected_reservoirs)-1, value=selected_reservoirs.index(RESERVOIR))

# def get_aec(reservoir, max_height):
#     aec_fp = Path(f'/tiger1/pdas47/tmsosPP/data/aec/srtm/{reservoir}.csv')
#     aec = pd.read_csv(aec_fp)
    
#     obs_aec_above_water = aec[aec['Elevation'] < max_height]
#     obs_aec_above_water = obs_aec_above_water.sort_values('Elevation')
#     obs_aec_above_water['CumArea_diff'] = obs_aec_above_water['CumArea'].diff()
#     obs_aec_above_water['z_score'] = (obs_aec_above_water['CumArea_diff'] - obs_aec_above_water['CumArea'].mean()) / obs_aec_above_water['CumArea'].std()
#     max_z_core_idx = obs_aec_above_water['z_score'].idxmax()
#     obs_aec_above_water = obs_aec_above_water.loc[max_z_core_idx:, :]
#     obs_aec_above_water = obs_aec_above_water[['Elevation', 'CumArea']]

#     return obs_aec_above_water

# def get_dam_elevations(
#         reservoir_id, 
#         percentile=1, 
#         merit_dem_elevations_path="/tiger1/pdas47/tmsosPP/data/dam_bottom_elevation/MERIT_DEM_1000_m_around_dam_locations.csv"
#     ):
#     merit_dem_elevations = pd.read_csv(merit_dem_elevations_path, dtype={'tmsos_id': str}).drop(
#         ['system:index', '.geo'], axis=1
#     )
#     elevations = merit_dem_elevations[merit_dem_elevations['tmsos_id'] == reservoir_id]
#     dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == reservoir_id]['DAM_HGT_M'].values[0])
#     dam_bottom = elevations[f'dem_p{percentile}'].values[0]
#     dam_top = dam_bottom + dam_height
#     return dam_bottom, dam_top


# def plot_elevations(index):
#     reservoir = selected_reservoirs[index]
#     dam_bottom, dam_top = get_dam_elevations(reservoir, percentile=1)
#     elevations = merit_dem_elevations[merit_dem_elevations['tmsos_id'] == reservoir]
#     min_elev = elevations['dem_min'].values[0]
#     max_elev = elevations['dem_max'].values[0]
#     elevation_percentile_cols = [
#         'dem_p1', 'dem_p2', 'dem_p3', 'dem_p4', 'dem_p5', 'dem_p6', 'dem_p7', 'dem_p8', 'dem_p9', 
#         'dem_p10', 'dem_p15', 'dem_p20', 'dem_p30', 'dem_p40', 'dem_p50', 'dem_p60', 'dem_p70', 
#         'dem_p80', 'dem_p90'
#     ]
#     elevation_at_percentile = [min_elev] + list(elevations[elevation_percentile_cols].values.flatten()) + [max_elev]
#     percentiles = [0] + [int(s.split('p')[-1]) for s in elevation_percentile_cols] + [100]

#     reservoir_name = res_names[reservoir]
#     dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == reservoir]['DAM_HGT_M'].values[0])

#     rectangle_x = 1  # 1st percentile
#     rectangle_y_start = dam_bottom
#     rectangle_y_end = rectangle_y_start + dam_height

#     rectangle = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#         color='green', alpha=0.5, line_width=2
#     )

#     return hv.Scatter((percentiles, elevation_at_percentile), 'Percentile', 'Elevation').opts(
#         height=400, width=500, title=f'{reservoir}: {reservoir_name}\nElevation Percentiles within 1000 m of dam location\nDam Height: {dam_height} m',
#         xlabel='Percentile', ylabel='Elevation (m)', size=10
#     ) * hv.Curve((percentiles, elevation_at_percentile)) * rectangle * hv.HLine(101) * hv.HLine(105)

# interactive_plot = pn.bind(plot_elevations, reservoir_selector)
# pn.Column(
#     reservoir_selector, interactive_plot
# )

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d8cbf6c-275a-4af3-bb0e-d3a6faa9b967': {'version…

The dam bottom elevation can be estimated by looking at a buffered region around the dam location.
The minimum elevation (0th percentile) within the region will represent the elevaiton of the downstream river.
However, the elevation of the river downstream may be lower than the elevation of reservoir bottom. Furthermore, the minimum value might be spurious in case of issues with the DEM data itself, due to height uncertainty.
Hence taking the 1st pecentile elevation as the reservoir bottom elevation may alleviate some of these issues. 

Using the minimum elevation, calculate the storage of the reservoir and compare it with the reported capacity in GRanD.

In [17]:
# elevations = merit_dem_elevations[merit_dem_elevations['tmsos_id'] == RESERVOIR]

# min_elev = elevations['dem_min'].values[0]
# dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
# if dam_height == -99:
#     print(f"Dam height unavailable. ")
#     dam_height = np.nan

# dam_bottom, dam_top = get_dam_elevations(RESERVOIR, percentile=1)

# print("Min max elevations: ", dam_bottom, dam_top)

# # Interpolate the max_area corresponding to the dam_top using np.interp
# max_area_interpolated = np.interp(dam_top, aec['Elevation'], aec['CumArea'])
# min_area = 0
# print("Min max areas: ", min_area, max_area_interpolated)

# obs_aec_above_water = get_aec(RESERVOIR, dam_top)

# # plot dam
# rectangle_x = 0
# rectangle_y_start = dam_bottom
# rectangle_y_end = dam_top

# dam_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='gray', alpha=0.5, line_width=2
# )

# obs_aec_above_water.hvplot(
#     x='CumArea', y='Elevation', label='SRTM'
# ) * hv.HLine(dam_bottom) * hv.HLine(dam_top) * dam_hv

Min max elevations:  226.8602091471354 449.8602091471354
Min max areas:  0 1076.8906813557942


:Overlay
   .Curve.SRTM   :Curve   [CumArea]   (Elevation)
   .HLine.I      :HLine   [x,y]
   .HLine.II     :HLine   [x,y]
   .Rectangles.I :Rectangles   [x0,y0,x1,y1]

In [18]:
# dam_bottom

226.8602091471354

In [19]:
# from scipy.integrate import cumulative_trapezoid

# def calculate_storage(aec_df):
#     """
#     Calculate the storage of a reservoir from its Area-Elevation Curve (AEC).

#     Parameters:
#     aec_df (pd.DataFrame): DataFrame containing 'Elevation' and 'CumArea' columns.

#     Returns:
#     pd.DataFrame: DataFrame with an additional 'Storage' column representing the storage in cubic meters.
#     """
#     elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

#     # cumulative_trapezoid takes two parameters.
#     # y = y-axis locations of points. these values will be integrated. 
#     # x = x-axis locations of points, where each y value is sampled. Area.
#     storage = cumulative_trapezoid(
#         elevation_normalized, 
#         aec_df['CumArea'] * 1e6
#     )
#     storage = np.insert(storage, 0, 0)

#     aec_df['Storage'] = storage
#     aec_df['Storage (mil. m3)'] = storage * 1e-6
#     return aec_df


In [20]:
# # generalized code for n-degree polynomial fitting

# import numpy as np
# import pandas as pd
# from scipy.optimize import minimize

# obs_aec_above_water = get_aec(RESERVOIR, dam_top)

# x = obs_aec_above_water['CumArea']
# y = obs_aec_above_water['Elevation']


# # Function to return predicted y-values from the polynomial
# def predict_y(params, x):
#     return np.polyval(params, x)

# # Objective function: residuals to minimize
# def objective(params, x, y):
#     predicted_y = predict_y(params, x)

#     return np.sum((predicted_y - y)**2)

# # Constraint 1: dy/dx > 0 -> derivative of the polynomial should be positive for all x
# def constraint_derivative(params):
#     # Sample points across the entire range of x (0 to np.max(x))
#     x_sample = np.linspace(0, np.max(x), 100)
    
#     # Get the derivative coefficients of the polynomial
#     derivative_coeffs = np.polyder(params[::-1], 1)  # Take the first derivative
    
#     # Evaluate the derivative at these sample points
#     derivative_values = np.polyval(derivative_coeffs, x_sample)  # np.polyder gives the derivative coefficients

#     # Return the minimum value of the derivative; it should be greater than 0
#     return derivative_values

# # Constraint 2: intercept should be within (dam_bottom, dam_bottom + 5)
# def constraint_intercept(params):
#     a0 = params[-1]  # Intercept is the last parameter (highest degree)
#     return a0 - dam_bottom, (dam_bottom + 5) - a0

# # Function to perform the minimization
# def fit_polynomial(x, y, degree):
#     # Initial guess for the polynomial parameters using np.polyfit
#     initial_guess = np.polyfit(x, y, degree)
#     print("Initial guess: ", initial_guess[::-1])
    
#     # Set up constraints
#     constraints = [
#         {'type': 'ineq', 'fun': lambda params: constraint_derivative(params)},  # dy/dx > 0
#         {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[0]},  # intercept within lower bound
#         {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[1]}   # intercept within upper bound
#     ]
    
#     # Perform minimization
#     result = minimize(objective, initial_guess, args=(x, y), constraints=constraints, options={'maxiter': 1000})
    
#     # Print results
#     print(f"Optimized polynomial for degree {degree}: {np.poly1d(result.x[::-1])}")
#     return result, initial_guess



# print("Dam bottom elevation: ", dam_bottom)
# print("Dam top elevation: ", dam_top)
# print("Dam capacity: ", capacity)


# result, initial_guess = fit_polynomial(x, y, 1)
# print(result)

Dam bottom elevation:  226.8602091471354
Dam top elevation:  449.8602091471354
Dam capacity:  36700.0
Initial guess:  [202.23705408   0.21589289]
Optimized polynomial for degree 1:  
226.9 x + 0.1906
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 4233.558459604783
       x: [ 1.906e-01  2.269e+02]
     nit: 4
     jac: [-1.612e+00  4.022e+01]
    nfev: 19
    njev: 4


below code works for 2 degree polynomial. `tmsos_2degpoly_aec_with_constraints`

In [21]:
# import numpy as np
# import pandas as pd
# from scipy.optimize import minimize

# # fit a 2° polynomial with constraints:
# # (1) df/dx > 0
# # (2) f(x) > 0 when x = 0

# poly_deg = 2

# obs_aec_above_water = get_aec(RESERVOIR, dam_top)
# # obs_aec_and_dam_bottom = pd.concat([pd.DataFrame({'Elevation': [dam_bottom], 'CumArea': [0]}), obs_aec_above_water])

# x = obs_aec_above_water['CumArea']
# y = obs_aec_above_water['Elevation']

# print("Dam bottom elevation: ", dam_bottom)
# print("Dam top elevation: ", dam_top)
# print("Dam capacity: ", capacity)

# # Function to return predicted y-values from the polynomial
# def predict_y(params, x):
#     a0, a1, a2 = params  # Only three parameters for a 2-degree polynomial
#     return a0 + a1 * x + a2 * x**2

# # Objective function: residuals to minimize
# def objective(params, x, y):
#     predicted_y = predict_y(params, x)
    
#     s = calculate_storage(pd.DataFrame({'CumArea': x, 'Elevation': predicted_y}))[['Storage (mil. m3)', 'Elevation']]
#     storage_at_dam_top = np.interp([dam_top], s['Elevation'], s['Storage (mil. m3)'])

#     return np.sum((predicted_y - y)**2)
#     # return ((capacity - storage_at_dam_top)**2) + np.sum((predicted_y - y)**2)/1e6
#     # return ((capacity - storage_at_dam_top)**2)

# # Constraint 1: dy/dx > 0 -> derivative of the polynomial should be positive for all x
# def constraint_derivative(params):
#     # Sample points across the entire range of x (0 to np.max(x))
#     x_sample = np.linspace(0, np.max(x), 100)
    
#     # Evaluate the derivative at these sample points
#     derivative_values = np.polyval(np.polyder(params[::-1]), x_sample)  # np.polyder gives the derivative coefficients

#     # Return the minimum value of the derivative; it should be greater than 0
#     return derivative_values

# # Constraint 2: intercept should be within (dam_bottom, dam_bottom + 5)
# def constraint_intercept(params):
#     a0 = params[0]  # Intercept is the first parameter
#     return a0 - dam_bottom, (dam_bottom + 5) - a0

# # Initial guess for the parameters [a0, a1, a2, a3]
# initial_guess = np.polyfit(x, y, 2)
# initial_guess = initial_guess[::-1] # reverse order to convert to scipy's format

# # initial_guess = [0.001, 0.001, 0.0001, 0.0001]
# print("initial guess: ", initial_guess)

# # Set up constraints
# constraints = [
#     {'type': 'ineq', 'fun': lambda params: constraint_derivative(params)},  # dy/dx > 0
#     # {'type': 'ineq', 'fun': lambda params: constraint_derivative(params, x)},  # dy/dx > 0
#     # {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)} # intercept greater than 0
#     {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[0]},
#     {'type': 'ineq', 'fun': lambda params: constraint_intercept(params)[1]}  
# ]

# # Perform minimization
# result = minimize(objective, initial_guess, args=(x, y), constraints=constraints, options={'maxiter': 1000})

# # Extract optimized parameters
# a0_opt, a1_opt, a2_opt = result.x  # Only three parameters for a 2-degree polynomial
# print(result)
# print(f"Optimized polynomial: y = {a0_opt} + {a1_opt}*x + {a2_opt}*x^2")

In [22]:
# x_pred = np.arange(0, max(x)) # area. 0-> max(area).
# y_pred = predict_y(result.x, x_pred)

# hv.Scatter((list(x_pred), list(y_pred)))

In [23]:
# if not result.success:
#     raise Exception("Failed to fit 1* polynomial")

In [24]:
# x_pred = np.arange(0, np.max(x), 0.25)
# y_pred = predict_y(result.x, x_pred)

# # Create a new DataFrame with the predicted x and y values
# predicted_df = pd.DataFrame({
#     'CumArea': x_pred, 
#     'Elevation': y_pred
# })

# # Clip the y-values (PredictedElevation) between dam_top+10 and dam_bottom-10
# predicted_df['ClippedElevation'] = predicted_df['Elevation'].clip(lower=dam_bottom - 10, upper=dam_top + 10)

# predicted_storage_df = calculate_storage(predicted_df)
# predicted_storage_df.head()

,CumArea,Elevation,ClippedElevation,Storage,Storage (mil. m3)
0,0.00,226.860209,226.860209,0.000000,0.000000
1,0.25,226.907868,226.907868,5957.340697,0.005957
2,0.50,226.955527,226.955527,23829.362789,0.023829
3,0.75,227.003185,227.003185,53616.066274,0.053616
4,1.00,227.050844,227.050844,95317.451155,0.095317


In [25]:
# # PLOT AEC
# rectangle_x = 0
# rectangle_y_start = dam_bottom
# rectangle_y_end = dam_top

# dam_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='green', alpha=0.5, line_width=2
# )

# aec_hv = hv.Scatter(
#     (predicted_df['CumArea'], predicted_df['ClippedElevation']), "Area", "Elevation", label='predicted Elevation'
# ) * hv.Scatter(
#     (0, dam_bottom), label='estimated dam bottom'
# ) * hv.Scatter(
#     (obs_aec_above_water['CumArea'], obs_aec_above_water['Elevation']), 'Area', 'Elevation', label='SRTM'
# ).opts(
#     color='black', height=300, width=600, legend_position='right', show_grid=True, 
#     title=f"AEC: {RESERVOIR}: {RESERVOIR_NAME}\nDam Height: {dam_height} m,\nEstimated Bottom Elevation: {dam_bottom:.2f} m"
# ) * dam_hv

# aec_hv

:Overlay
   .Scatter.Predicted_Elevation  :Scatter   [Area]   (Elevation)
   .Scatter.Estimated_dam_bottom :Scatter   [x]   (y)
   .Scatter.SRTM                 :Scatter   [Area]   (Elevation)
   .Rectangles.I                 :Rectangles   [x0,y0,x1,y1]

In [26]:
# volume_at_dam_top = predicted_storage_df.iloc[-1]['Storage (mil. m3)']
# volume_at_dam_top

# print(f"TOTAL calculated storage if filled to brim: {volume_at_dam_top:.2f} mil. m3;")
# print(f"Reported capacity: {capacity:.2f} Mil. m3")

# # Interpolate to find the elevation corresponding to the reported capacity
# elevation_at_capacity = np.interp(capacity, predicted_storage_df['Storage (mil. m3)'], predicted_storage_df['Elevation'])
# area_at_capacity = np.interp(elevation_at_capacity, predicted_storage_df['Elevation'], predicted_storage_df['CumArea'])

# print(f"Elevation at reported capacity: {elevation_at_capacity:.2f} m")
# print(f"Area at reported capacity: {area_at_capacity:.2f} km²")

TOTAL calculated storage if filled to brim: 110304.98 mil. m3;
Reported capacity: 36700.00 Mil. m3
Elevation at reported capacity: 345.15 m
Area at reported capacity: 620.51 km²


In [27]:
# # Plot dam as a rectangle
# rectangle_x = 0
# rectangle_y_start = dam_bottom
# rectangle_y_end = volume_at_dam_top

# dam_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='gray', alpha=0.5, line_width=2
# )
# # Calculate max area at dam top as a separate variable
# max_area_at_dam_top = np.interp(dam_top, predicted_storage_df['Elevation'], predicted_storage_df['CumArea'])

# aev_hv = dam_hv * predicted_storage_df.hvplot(
#     x='CumArea', y='Storage (mil. m3)', label='Predicted Storage'
# ).opts(
#     show_grid=True, 
#     ylim=(0, capacity + capacity * 0.1), 
#     xlim=(0, max_area_at_dam_top),  # Set xlim as 0 to max area at dam top
#     title=f"{RESERVOIR} - {RESERVOIR_NAME}: Storage, calculated and reported",
#     xlabel='Cumulative Area (km²)',
#     ylabel='Storage (Million m³)'
# ) * hv.HLine(capacity).opts(
#     color='black', line_dash='dashed'
# ) * hv.Text(
#     x=0, y=capacity, text=f'Reported Capacity: {capacity:.1f} Mil. m3', halign='left', valign='bottom'
# ) * hv.Text(
#     x=0, y=volume_at_dam_top, text=f'Total Calculated Storage: {volume_at_dam_top:.1f} Mil. m3', halign='left', valign='top'
# )

# # Add another plot with elevation on the x-axis
# elevation_plot = predicted_storage_df.hvplot(
#     x='Elevation', y='Storage (mil. m3)', label='Storage vs Elevation'
# ).opts(
#     show_grid=True,
#     title=f"{RESERVOIR} - {RESERVOIR_NAME}: Storage vs Elevation",
#     xlabel='Elevation (m)',
#     ylabel='Storage (Million m³)',
#     xlim=(min_elev, dam_top)
# )

# # Plot dam as a rectangle in the elevation-storage plot
# rectangle_x = min_elev
# rectangle_y_start = 0
# rectangle_y_end = volume_at_dam_top

# dam_storage_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='gray', alpha=0.5, line_width=2
# )

# elevation_plot = elevation_plot * dam_storage_hv

# aev_hv = (aev_hv + elevation_plot).cols(1)

# aev_hv

:Layout
   .Overlay.I  :Overlay
      .Rectangles.I            :Rectangles   [x0,y0,x1,y1]
      .Curve.Predicted_Storage :Curve   [CumArea]   (Storage (mil. m3))
      .HLine.I                 :HLine   [x,y]
      .Text.I                  :Text   [x,y]
      .Text.II                 :Text   [x,y]
   .Overlay.II :Overlay
      .Curve.Storage_vs_Elevation :Curve   [Elevation]   (Storage (mil. m3))
      .Rectangles.I               :Rectangles   [x0,y0,x1,y1]

In [28]:
# # save storage df
# # Save the predicted storage dataframe with optimization result as comments
# with open(f'/tiger1/pdas47/tmsosPP/data/aec/aev_1deg_polynomial_with_constraints/{RESERVOIR}.csv', 'w') as f:
#     f.write("# Optimization result:\n")
#     for line in str(result).split('\n'):
#         f.write(f"# {line}\n")
#     f.write("# Initial guess:\n")
#     for line in str(initial_guess).split('\n'):
#         f.write(f"# {line}\n")
#     predicted_storage_df.to_csv(f, index=False)

# # save aev plot
# hv.save(aev_hv, f"/tiger1/pdas47/tmsosPP/results/figures/aev_1deg_polynomial_with_constraints/aev/{RESERVOIR}.png", fmt='png')
# hv.save(aec_hv, f"/tiger1/pdas47/tmsosPP/results/figures/aev_1deg_polynomial_with_constraints/aec/{RESERVOIR}.png", fmt='png')

FileNotFoundError: [Errno 2] No such file or directory: '/tiger1/pdas47/tmsosPP/data/aec/aev_1deg_polynomial_with_constraints/1078.csv'

In [27]:
# # fit a 2° polynomial. by adding the dam bottom elevation as an extra point in the observed SRTM dem.

# from sklearn.linear_model import Ridge
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import PolynomialFeatures, SplineTransformer

# poly_deg = 2

# pft = PolynomialFeatures(degree=poly_deg)
# pft_pipeline = make_pipeline(
#     pft,
#     Ridge(alpha=1e-3),
# )

# prediction_range = np.linspace(
#     dam_bottom, dam_top, int(dam_top - dam_bottom)
# ).reshape(-1, 1)

# dam_bottom, dam_top = get_dam_elevations(RESERVOIR, percentile=1)

# obs_aec_above_water = get_aec(RESERVOIR, dam_top)
# obs_aec_and_dam_bottom = pd.concat([pd.DataFrame({'Elevation': [dam_bottom], 'CumArea': [0]}), obs_aec_above_water])

# poly_predictions = pft_pipeline.fit(
#     obs_aec_and_dam_bottom[['Elevation']], obs_aec_and_dam_bottom[['CumArea']]).predict(
#     pd.DataFrame(prediction_range, columns=['Elevation'])
# )

# poly_pred_df = pd.DataFrame({
#     'Elevation': prediction_range.flatten(),
#     'CumArea': poly_predictions.flatten()
# })

# poly_pred_df

# # PLOT
# print("Min max elevations: ", dam_bottom, dam_top)
# print("Min max areas: ", min_area, max_area_interpolated)


# # plot dam
# rectangle_x = 0
# rectangle_y_start = dam_bottom
# rectangle_y_end = dam_top

# dam_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='gray', alpha=0.5, line_width=2
# )

# obs_aec_hv = obs_aec_and_dam_bottom.hvplot(
#     x='CumArea', y='Elevation', label='SRTM + dam bottom added'
# ) 
# poly_pred_hv = poly_pred_df.hvplot(
#     x='CumArea', y='Elevation', label=f'{poly_deg} deg polynomial'
# )

# aec_hv = obs_aec_hv * poly_pred_hv * hv.HLine(dam_bottom).opts(color='gray') * hv.HLine(dam_top).opts(color='gray') * dam_hv
# aec_hv = aec_hv.opts(
#     title=f"AEC: {RESERVOIR}: {RESERVOIR_NAME}"
# )
# aec_hv

In [28]:
# from scipy.integrate import cumulative_trapezoid

# def calculate_storage(aec_df):
#     """
#     Calculate the storage of a reservoir from its Area-Elevation Curve (AEC).

#     Parameters:
#     aec_df (pd.DataFrame): DataFrame containing 'Elevation' and 'CumArea' columns.

#     Returns:
#     pd.DataFrame: DataFrame with an additional 'Storage' column representing the storage in cubic meters.
#     """
#     elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

#     # cumulative_trapezoid takes two parameters.
#     # y = y-axis locations of points. these values will be integrated. 
#     # x = x-axis locations of points, where each y value is sampled. Area.
#     storage = cumulative_trapezoid(
#         elevation_normalized, 
#         aec_df['CumArea'] * 1e6
#     )
#     storage = np.insert(storage, 0, 0)

#     aec_df['Storage'] = storage
#     aec_df['Storage (mil. m3)'] = storage * 1e-6
#     return aec_df

# # Example usage:
# storage_poly_pred_df = calculate_storage(poly_pred_df)
# storage_poly_pred_df.head()

In [29]:
# volume_at_dam_top = storage_poly_pred_df.iloc[-1]['Storage (mil. m3)']

# print(f"TOTAL calculated storage if filled to brim: {volume_at_dam_top:.2f} mil. m3;")
# print(f"Reported capacity: {capacity:.2f} Mil. m3")

# # Interpolate to find the elevation corresponding to the reported capacity
# elevation_at_capacity = np.interp(capacity, storage_poly_pred_df['Storage (mil. m3)'], storage_poly_pred_df['Elevation'])
# area_at_capacity = np.interp(elevation_at_capacity, storage_poly_pred_df['Elevation'], storage_poly_pred_df['CumArea'])

# print(f"Elevation at reported capacity: {elevation_at_capacity:.2f} m")
# print(f"Area at reported capacity: {area_at_capacity:.2f} km²")

In [30]:
# # Plot dam as a rectangle
# rectangle_x = 0
# rectangle_y_start = dam_bottom
# rectangle_y_end = volume_at_dam_top

# dam_hv = hv.Rectangles([(rectangle_x - 0.5, rectangle_y_start, rectangle_x + 0.5, rectangle_y_end)]).opts(
#     color='gray', alpha=0.5, line_width=2
# )

# aev_hv = capacity_hv * storage_poly_pred_df.hvplot(
#     x='CumArea', y='Storage (mil. m3)'
# ).opts(xlabel='CumArea')

# aev_hv = dam_hv * aev_hv.opts(
#     title = f"AEV: {RESERVOIR} - {RESERVOIR_NAME}\nVolume if filled to dam top: {volume_at_dam_top:.2f} mil. m3\nCapacity: {capacity:.2f} mil. m3\nArea at capacity: {area_at_capacity:.2f} km2, Elevation at capacity: {elevation_at_capacity:.2f} m",
# )
# aev_hv

In [31]:
# aev_hv_mod = hv.HLine(7500).opts(color='red') * hv.HLine(5700) * storage_poly_pred_df.hvplot(
#     x='CumArea', y='Storage (mil. m3)'
# ).opts(xlabel='CumArea') * hv.VLine(0).opts(color='gray') * hv.VLine(118) * obs_aec_above_water.hvplot(x='CumArea', y='Elevation', kind='scatter')

# aev_hv_mod = aev_hv_mod.opts(
#     title = f"AEV: {RESERVOIR} - {RESERVOIR_NAME}\nVolume if filled to dam top: {volume_at_dam_top:.2f} mil. m3\nCapacity: {capacity:.2f} mil. m3\nArea at capacity: {area_at_capacity:.2f} km2, Elevation at capacity: {elevation_at_capacity:.2f} m",
# )
# aev_hv_mod

In [32]:
# # save storage df
# predicted_storage_df.to_csv(f'/tiger1/pdas47/tmsosPP/data/aec/aev_2deg_polynomial_with_constraints/{RESERVOIR}.csv', index=False)

# # save aev plot
# hv.save(aev_hv, f"/tiger1/pdas47/tmsosPP/results/figures/aev_2deg_polynomial_with_constraints/aev/{RESERVOIR}.png", fmt='png')
# hv.save(aec_hv, f"/tiger1/pdas47/tmsosPP/results/figures/aev_2deg_polynomial_with_constraints/aec/{RESERVOIR}.png", fmt='png')

In [33]:
# print(f"Interpolated max_area at dam_top ({dam_top:.2f} m): {max_area_interpolated:.2f} km²")

<!-- We know the reported capcaity of the reservoir. Let's call it $S$.
Let's assume that the function that best defines the AEC of the reservoir is a power function, which is monotonically increasing. With increasing Area, the Elevation must increase, and vice versa. The function can be of the form $E = aA^b$ where $E$ is the elevation, $A$ is the area, and $a$ and $b$ are constants. Let's call this function $AEC_{model}$.

We can calculate the storage of the reservoir using this AEC by integrating it. Let's call this calculated storage $s$.

The error in the reported capacity is given by $e = S - s$

Furthermore, we also have the observed AEC of the reservoir observed by SRTM. Let's call this observed part of the AEC $AEC_{obs}$

Since the AECs are arrays of values, we can find the error in the modeled and observed AEC by calculating the root mean squared error (RMSE) between the two. $RMSE = sqrt(MSE(AEC_{obs}, AEC_{model}))$

These two error can be minimized to get $a$ and $b$. -->

In [34]:
# capacity

In [35]:
# from scipy.optimize import minimize
# from scipy.integrate import trapezoid

# def s(params, E):
#     a = params[0]
#     b = params[1]
#     c = params[2]
    
#     A = a + (b * E) + (c * np.power(E, 2))

#     # storage = trapezoid(
#     #     E, 
#     #     A
#     # )

#     return A

# elevations = np.arange(dam_bottom, dam_top) # 1 km2 interval

# params = [
#     -954, 17.9, -0.0779
# ]
# areas = s(
#     params, elevations
# )

# hv.Scatter(
#     (areas, elevations), 'area', 'elevation'
# )
# # f = lambda params, elevations: capacity - s(params, elevations)

# # # capacity - s(x, 0.1, 1)
# # res = minimize(
# #     f, params, method='nelder-mead', args=(elevations), options={'disp': True}
# # )

In [36]:
# hv.Scatter(
#     (areas, elevations), 'area', 'elevation'
# ) * aec.hvplot(x='CumArea', y='Elevation')

In [37]:
# capacity

In [38]:
# # params = res.x
# params = [-954, 17.9, -0.0779]

# areas = (params[0] + params[1] * elevations + params[2] * elevations**2) * 1e-6

# aec_mod_without_zero_area.hvplot(
#     x='Elevation', y='CumArea'
# ) * hv.Scatter((elevations, areas), 'Elevation', 'Area')

In [39]:
# aec_mod_without_zero_area

<!-- ## fill AEC using polynomial interpolation -->

In [40]:
# aec_mod = aec_mod.reset_index(drop=True)[['Elevation', 'CumArea']]

# aec_mod.head(2)

In [41]:
# from sklearn.linear_model import Ridge
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import PolynomialFeatures, SplineTransformer


# pft = PolynomialFeatures(degree=poly_deg)
# pft_pipeline = make_pipeline(
#     pft,
#     Ridge(alpha=1e-3),
# )

In [42]:
# prediction_range = np.linspace(
#     min_height_inferred, max_height_inferred, int(max_height_inferred - min_height_inferred)
# ).reshape(-1, 1)
# poly_predictions = pft_pipeline.fit(aec_mod[['Elevation']], aec_mod[['CumArea']]).predict(
#     pd.DataFrame(prediction_range, columns=['Elevation'])
# )

# poly_pred_df = pd.DataFrame({
#     'Elevation': prediction_range.flatten(),
#     'CumArea': poly_predictions.flatten()
# })
# poly_pred_df['CumArea'] = poly_pred_df['CumArea'].clip(0)
# if (poly_pred_df['CumArea'] == 0).sum() > 0:
#     zero_elevation = poly_pred_df[poly_pred_df['CumArea'] == 0]['Elevation'].max()
# else:
#     zero_elevation = min_height_inferred
# poly_pred_df = poly_pred_df[poly_pred_df['Elevation'] >= zero_elevation]
# poly_pred_df.hvplot(x='Elevation', y='CumArea')

# aec_mod_without_zero_area = aec_mod.copy()
# aec_mod_without_zero_area = aec_mod_without_zero_area[aec_mod_without_zero_area['CumArea']!=0]
# elevations = np.linspace(zero_elevation, max_height_inferred, int(max_height_inferred-zero_elevation))
# areas = [
#     np.interp(elevation, aec_mod_without_zero_area['Elevation'], aec_mod_without_zero_area['CumArea']) if np.logical_and(
#         elevation >= aec_mod_without_zero_area['Elevation'].min(), elevation <= aec_mod_without_zero_area['Elevation'].max()
#     ) else np.interp(elevation, poly_pred_df['Elevation'], poly_pred_df['CumArea']) for elevation in elevations
# ]
# obs_extrapolated = [
#     'SRTM' if np.logical_and(
#         elevation >= aec_mod_without_zero_area['Elevation'].min(), elevation <= aec_mod_without_zero_area['Elevation'].max()
#     ) else 'extrapolated' for elevation in elevations
# ]

# extrapolated_aec = {
#     'Elevation': elevations,
#     'CumArea': areas,
#     'obs_or_extrapolated': obs_extrapolated
# }

# # extrapolated_aec
# extrapolated_aec = pd.DataFrame(extrapolated_aec)
# if len(extrapolated_aec[extrapolated_aec['CumArea'] == 0]) == 0:
#     zero_area_elev = extrapolated_aec.iloc[0]['Elevation']
# else:
#     zero_area_elev = extrapolated_aec.loc[extrapolated_aec[extrapolated_aec['CumArea'] == 0].idxmax()['Elevation']]['Elevation']


# extrapolated_aec = extrapolated_aec[extrapolated_aec['Elevation'] >= zero_area_elev]

# # remove any extrapolated values above the SRTM observed elevation
# idx_max_extrapolated_value = extrapolated_aec[extrapolated_aec['obs_or_extrapolated'] == 'extrapolated'].idxmax()['Elevation']
# # check if the previous value is SRTM, if yes, delete the extrapolated value
# if extrapolated_aec.loc[idx_max_extrapolated_value-1, 'obs_or_extrapolated']:
#     print("deleting extrapolated point above observed AEC")
#     extrapolated_aec = extrapolated_aec.iloc[:-1]

# # PLOT

# min_inferred_elevation_hv = hv.HLine(min_height_inferred).opts(color='orange')
# max_inferred_elevation_hv = hv.HLine(max_height_inferred).opts(color='orange')

# extrapolated_aec.hvplot.scatter(
#     x='CumArea', y='Elevation', by='obs_or_extrapolated'
# ).opts(
#     height=400, width=500, title=f'{RESERVOIR}: {RESERVOIR_NAME}\nExtrapolated AEC',
#     xlabel='Area (km2)', ylabel='Elevation (m)'
# ) * min_inferred_elevation_hv * max_inferred_elevation_hv

In [43]:
# srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/srtm_extrapolated')

# # poly_pred_df.round(2).to_csv(srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}.csv', index=False)
# # extrapolated_aec.round(2).to_csv(srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}.csv', index=False)
# # print(f"Saved at {srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}.csv'}")

In [44]:
# val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
# val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

# # aec_fp = srtm_extrapolated_dir / f'{RESERVOIR}_poly_{poly_deg}.csv'
# # aec_df = pd.read_csv(aec_fp)

# from scipy.integrate import cumulative_trapezoid
# # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html#scipy.integrate.cumulative_trapezoid
# ## Cumulatively integrate y(x) using the composite trapezoidal rule.


# aec_df = extrapolated_aec

# elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

# # cumulative_trapezoid takes two parameters.
# # y = y-axis locations of points. these values will be integrated. 
# # https://en.wikipedia.org/wiki/File:Composite_trapezoidal_rule_illustration.png. Normalized Elevation.
# # x = x-axis locations of points, where each y value is sampled. Area.
# storage = cumulative_trapezoid(
#     elevation_normalized, 
#     aec_df['CumArea'] * 1e6
# )
# storage = np.insert(storage, 0, 0)

# aec_df['Storage'] = storage
# aec_df['Storage (mil. m3)'] = storage * 1e-6
# aec_df

In [45]:
# aec_df.hvplot(x='CumArea', y='Elevation').opts(height=300, width=400, title=f'{RESERVOIR}: {RESERVOIR_NAME}  [A-E]', ylabel='Elevation (m)', xlabel='Area (km2)') \
# + (aec_df.hvplot(x='Elevation', y='Storage (mil. m3)', title=f'{RESERVOIR}: {RESERVOIR_NAME}  [S-E]').opts(height=300, width=400, ylabel='Storage (Million m3)', xlabel='Elevation (m)') * capacity_hv) \
# + (aec_df.hvplot(x='CumArea', y='Storage (mil. m3)', title=f'{RESERVOIR}: {RESERVOIR_NAME}  [S-A]').opts(height=300, width=400, ylabel='Storage (Million m3)', xlabel='Area (km2)') * capacity_hv)

In [46]:
# # save aec in `srtm_extrapolated_storage`
# srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/srtm_extrapolated_storage/')